### Introduction to TensorFlow

#### Graph execution (standard)

In [1]:
import tensorflow as tf
from datetime import datetime
print tf.executing_eagerly()

tf.reset_default_graph()

# log file for tensorboard
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
logdir = 'tf_logs/run-'+str(now)

# setup the execution graph to calculate an inner product of two tensors
x = tf.Variable([[1.,2.,3.]])
y = tf.Variable([[4.,5.,6.]])
yt = tf.transpose(y)
z = tf.matmul(x,yt)
init = tf.global_variables_initializer()
file_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())   

# execute in the default session
sess = tf.Session()
sess.run(init)
result = sess.run(z)
print result
sess.close()
file_writer.close()

False
[[32.]]


In [8]:
# inspect the graph on localhost:6006
!tensorboard --logdir tf_logs/

TensorBoard 1.12.2 at http://vivo:6006 (Press CTRL+C to quit)
^C


In [3]:
# or do the same calculation this way
with tf.Session() as sess:
    init.run()
    result = z.eval() 
print result   

[[32.]]


In [4]:
# or this way
import numpy as np

# input data as numpy arrays
x_data = np.array([[1,2,3]])
y_data = np.array([[4,5,6]])

# execution graph with placeholders instead of variables
x = tf.placeholder(tf.float32,shape=(1,3))
y = tf.placeholder(tf.float32,shape=(1,3))
yt = tf.transpose(y)
z = tf.matmul(x,yt)

# execute in default session
with tf.Session() as sess:
    result = z.eval(feed_dict = {x: x_data, y: y_data})
print result

[[32.]]


In [5]:
# same as above, but this time we save the model

saver = tf.train.Saver() # saver node

with tf.Session() as sess:
    init.run()
    result = z.eval(feed_dict = {x: x_data, y: y_data})
    save_path = saver.save(sess,'imagery/model.ckpt') 
print result

[[32.]]


In [6]:
# restore the session and run it again

with tf.Session() as sess:
    saver.restore(sess,'imagery/model.ckpt')
    result = z.eval(feed_dict={x:x_data, y:y_data} )
print result

INFO:tensorflow:Restoring parameters from imagery/model.ckpt
[[32.]]


#### Automatic differentiation

In [7]:
tf.reset_default_graph()

# log file for tensorboard
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
logdir = 'tf_logs/run-'+str(now)

# a 2x2 tensor of ones
x = tf.Variable(tf.ones((2,2)))

# a computation z = sum(x)^2 = 16
y = tf.reduce_sum(x)
z = y*y

# Derivative of z with respect to the original input tensor x
#     dz/dx_ij = 2*sum(x)*d(x11+x12+x21+x22)/dx_ij = 2*4*1 = 8, for i,j = 1,2
# Derivative of z with respect to y = sum(x)
#     dz/sum(x) = 2*sum(x) = 2*4 = 8
result = tf.gradients(z,[x,y])

file_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())

# run the calculation
with tf.Session() as sess:
    x.initializer.run()
    print 'z = %s' %z.eval()
    print 'dz/dx = \n%s' %result[0].eval()
    print 'dz/dy = %s' %result[1].eval()


z = 16.0
dz/dx = 
[[8. 8.]
 [8. 8.]]
dz/dy = 8.0


#### Eager execution (RESTART THE IPYTHON KERNEL!!)

In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
print tf.executing_eagerly()

True


In [2]:
x = np.mat([1.,2.,3.])
y = np.mat([4.,5.,6.])

# in numpy
z = x*y.T
print 'numpy result: %f' %z

# in tensorflow
yt = tf.transpose(y)
z = tf.matmul(x,yt)
print 'tensor result: %s' %z

# convert tensor result to numpy array
print 'convert tensor to numpy: %f' %z.numpy()

# where is the variable z?
print 'location of z: %s' % z.device

numpy result: 32.000000
tensor result: tf.Tensor([[32.]], shape=(1, 1), dtype=float64)
convert tensor to numpy: 32.000000
location of z: /job:localhost/replica:0/task:0/device:CPU:0


#### Automatic differentiation with eager execution

In [3]:
# a 2x2 tensor, now we can print it without a session
x = tf.ones((2, 2))
print 'as tensor x = %s' % x
print 'as numpy array x =\n %s' % x.numpy()

as tensor x = tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)
as numpy array x =
 [[1. 1.]
 [1. 1.]]


In [ ]:
with tf.GradientTape(persistent=True) as t:
  t.watch(x)
# a computation z = sum(x)^2 
  y = tf.reduce_sum(x)   # sum(x)
  z = tf.multiply(y, y)  # sum(x)^2
print 'z = sum(x)^2 = %s' % z.numpy()

In [ ]:
# Derivative of z with respect to the original input tensor x
#     dz/dx_ij = 2*sum(x)*d(x11+x12+x21+x22)/dx_ij = 2*4*1 = 8, for i,j = 1,2
dz_dx = t.gradient(z, x)
print 'dz/dx =\n %s' % dz_dx.numpy()

In [ ]:
# Derivative of z with respect to sum(x)
#     dz/sum(x) = 2*sum(x) = 2*4 = 8
dz_dy = t.gradient(z, y)
print 'dz/d(sum(x)) = %s' % dz_dy.numpy()

In [ ]:
# the tape object is still im memory
print t
# so delete it
del t
# confirm
try: 
    print t
except:
    print 'gone'